In [21]:
import torch
import torch.nn as nn
import math
max_period = 100
dim = 10
half = dim // 2
freqs = torch.exp (
    # divide by half at the end so that we dont get absurdly small numebrs
    -math.log(max_period) * (torch.arange(start=0, end=half, dtype=torch.float32, device='cuda'))/half
)

freqs.shape

# 4
# -2 * (0 , 1, 2, 3)
# (0, -2, -4, -6) / 4
# (0, -0.5, -1, -1.5)

torch.Size([5])

In [22]:
t = torch.tensor([5])

t[:, None].shape, t.shape, t.unsqueeze(1).shape, torch.allclose(t[:, None], t.unsqueeze(1)), freqs.shape

(torch.Size([1, 1]),
 torch.Size([1]),
 torch.Size([1, 1]),
 True,
 torch.Size([5]))

In [23]:
#[1,1] * [5]
scaled_sigma_t = t.unsqueeze(1).to('cuda') * freqs
without_extra_dim = t.to('cuda') * freqs
scaled_sigma_t.shape, without_extra_dim.shape

(torch.Size([1, 5]), torch.Size([5]))

In [24]:
scaled_sigma_t, scaled_sigma_t.shape, freqs, t

(tensor([[5.0000, 1.9905, 0.7924, 0.3155, 0.1256]], device='cuda:0'),
 torch.Size([1, 5]),
 tensor([1.0000, 0.3981, 0.1585, 0.0631, 0.0251], device='cuda:0'),
 tensor([5]))

In [25]:
# now my arguments to sinusoids are 
args = scaled_sigma_t

embedding = torch.cat((torch.cos(args), torch.sin(args)), dim=-1)
assert embedding.shape[-1] == 10

args.shape, embedding.shape

(torch.Size([1, 5]), torch.Size([1, 10]))

In [26]:
torch.zeros_like(embedding[:, :1])

tensor([[0.]], device='cuda:0')

In [27]:
# append bias term (0 freq component)
torch.allclose(torch.zeros_like(embedding[:, 0]), torch.zeros_like(embedding[:, :1]))

True

In [28]:
scaled_sigma_t = t.unsqueeze(1).to('cuda') * freqs.to('cuda')
bullshit = t[:, None].float().to('cuda') * freqs[None].to('cuda')
torch.allclose(scaled_sigma_t, bullshit)

True

In [30]:
class TimestepEmbedder(nn.Module):
    """Embeds scalar timesteps into vector representations.
    
    Args:
        hidden_size (int): Size of hidden dimension
        act_layer (Any): Activation layer constructor
        frequency_embedding_size (int, 512): Size of frequency embedding
    """
    def __init__(
        self,
        hidden_size: int,
        act_layer,
        frequency_embedding_size: int = 512
    ):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(frequency_embedding_size, hidden_size, bias=True),
            act_layer,
            nn.Linear(hidden_size, hidden_size, bias=True),
        )
        self.frequency_embedding_size = frequency_embedding_size

    def forward(self, x):
        #print (self.dtype)
        return x

In [31]:
a = TimestepEmbedder(1, nn.GELU(), 512)
a(1)

1

In [32]:
type(a)

__main__.TimestepEmbedder